In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Mounted at /content/drive


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9601
Train Precision: 0.9602
Train Recall: 0.9601
Train F1 Score: 0.9598
Train Confusion Matrix:
[[ 30   1   1]
 [  0 120  12]
 [  0   5 307]]

Test Accuracy: 0.7268
Test Precision: 0.7423
Test Recall: 0.7268
Test F1 Score: 0.7328
Test Confusion Matrix:
[[  3   7   3]
 [  3  22  16]
 [  5  22 124]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      96.01%
Test Accuracy       72.68%
Train-Test Gap      23.33%
ROC-AUC             0.71
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 0.5, 'n_estimators': 150, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7512
Test Precision: 0.7073
Test Recall: 0.7512
Test F

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=30. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'n_estimators': 50, 'learning_rate': 1.0}
AdaBoost Model Metrics:
Train Accuracy: 0.7857
Train Precision: 0.7914
Train Recall: 0.7857
Train F1 Score: 0.7866
Train Confusion Matrix:
[[ 20   9   3]
 [  2  87  43]
 [  0  45 267]]

Test Accuracy: 0.7024
Test Precision: 0.7112
Test Recall: 0.7024
Test F1 Score: 0.7001
Test Confusion Matrix:
[[  2   7   4]
 [  1  18  22]
 [  1  26 124]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      78.57%
Test Accuracy       70.24%
Train-Test Gap      8.33%
ROC-AUC             0.65
Overfitting         No


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8887
Train Precision: 0.8912
Train Recall: 0.8887
Train F1 Score: 0.8837
Train Confusion Matrix:
[[ 19   5   8]
 [  0  99  33]
 [  1   6 305]]

Test Accuracy: 0.7561
Test Precision: 0.7314
Test Recall: 0.7561
Test F1 Score: 0.7406
Test Confusion Matrix:
[[  2   7   4]
 [  2  17  22]
 [  2  13 136]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      88.87%
Test Accuracy       75.61%
Train-Test Gap      13.26%
ROC-AUC             0.69
Overfitting         Yes


Running SVM model...
Best Params: {'kernel': 'linear', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9139
Train Precision: 0.9158
Train Recall: 0.9139
Train F1 Score: 0.9114
Train Confusion Matrix:
[[ 29   1   2]
 [  0 101  31]
 [  1   6 305]]

Test Accuracy: 0.7415
Test Precision: 0.7311
Test Recall: 0.7415
Test F1 Score: 0.7355
Test Confusion Ma

In [ ]:
from google.colab import drive
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.decomposition import PCA

# Mount Google Drive
drive.mount('/content/drive')

# Load RoBERTa tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
model = AutoModel.from_pretrained('roberta-base')

# Generate RoBERTa sentence embeddings
def generate_roberta_embeddings(sentences):
    sentence_vectors = []
    for sentence in sentences:
        inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            # Use the [CLS] token's embedding
            cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        sentence_vectors.append(cls_embedding)
    return np.array(sentence_vectors)

def apply_pca(X, variance_threshold=0.95):
    pca = PCA()
    X_reduced = pca.fit_transform(X)

    # Determine number of components to preserve desired variance
    explained_variance = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(explained_variance >= variance_threshold) + 1
    print(f"Number of components selected: {n_components}")

    # Reapply PCA with the determined number of components
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    return X_reduced, pca

# Performance table function
def performance_table(y_train, train_pred, y_test, test_pred, model_name):
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    train_test_gap = abs(train_acc - test_acc)
    # Get unique labels from both y_test and test_pred
    all_labels = sorted(list(set(y_test) | set(test_pred)))
    # Ensure both one-hot encoded arrays have the same columns
    y_test_dummies = pd.get_dummies(y_test, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    test_pred_dummies = pd.get_dummies(test_pred, columns=all_labels).reindex(columns=all_labels, fill_value=0)
    roc_auc = roc_auc_score(y_test_dummies, test_pred_dummies, multi_class="ovr", average="weighted")

    overfitting = "Yes" if train_test_gap > 0.1 else "No"
    print(f"\nPerformance Table for {model_name}:")
    print(f"{'Metric':<20}{'Value':<10}")
    print(f"{'-'*30}")
    print(f"{'Train Accuracy':<20}{train_acc:.2%}")
    print(f"{'Test Accuracy':<20}{test_acc:.2%}")
    print(f"{'Train-Test Gap':<20}{train_test_gap:.2%}")
    print(f"{'ROC-AUC':<20}{roc_auc:.2f}")
    print(f"{'Overfitting':<20}{overfitting}")
    print("\n")


def print_metrics(y_true, y_pred, dataset_type="Test"):
    print(f"{dataset_type} Accuracy: {accuracy_score(y_true, y_pred):.4f}")
    print(f"{dataset_type} Precision: {precision_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Recall: {recall_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} F1 Score: {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
    print(f"{dataset_type} Confusion Matrix:\n{confusion_matrix(y_true, y_pred)}\n")

# Hyperparameter tuning function
def tune_model(model, param_distributions, X_train, y_train):
    random_search = RandomizedSearchCV(model, param_distributions, n_iter=30, cv=5, scoring='f1_weighted', random_state=42, n_jobs=-1)
    random_search.fit(X_train, y_train)
    print(f"Best Params: {random_search.best_params_}")
    return random_search.best_estimator_

# Individual model functions with hyperparameter tuning
def knn_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_neighbors': range(1, 21), 'weights': ['uniform', 'distance']}
    model = tune_model(KNeighborsClassifier(), param_grid, X_train, y_train)
    print("KNN Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "KNN")

def random_forest_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
    model = tune_model(RandomForestClassifier(), param_grid, X_train, y_train)
    print("Random Forest Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Random forest")

def decision_tree_model(X_train, y_train, X_test, y_test):
    # Expanded parameter grid
    param_grid = {
        'max_depth': [None, 10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4, 6, 8]
    }
    model = tune_model(DecisionTreeClassifier(), param_grid, X_train, y_train)
    print("Decision Tree Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Decision Tree")

def naive_bayes_model(X_train, y_train, X_test, y_test):
    model = GaussianNB()
    model.fit(X_train, y_train)
    print("Naive Bayes Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")

def mlp_model(X_train, y_train, X_test, y_test):
    param_grid = {'hidden_layer_sizes': [(50,), (100,)], 'activation': ['relu', 'tanh'], 'solver': ['adam', 'sgd']}
    model = tune_model(MLPClassifier(max_iter=300), param_grid, X_train, y_train)
    print("MLP Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "MLP")

def xgboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.5, 0.8, 1.0]}
    model = tune_model(XGBClassifier(eval_metric='mlogloss'), param_grid, X_train, y_train)
    print("XGBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "XG BOOST")

def adaboost_model(X_train, y_train, X_test, y_test):
    param_grid = {'n_estimators': [50, 100, 150], 'learning_rate': [0.01, 0.1, 1.0]}
    model = tune_model(AdaBoostClassifier(algorithm='SAMME'), param_grid, X_train, y_train)
    print("AdaBoost Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Ada Boost")

def logistic_regression_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.01, 0.1, 1, 10], 'penalty': ['l2'], 'solver': ['lbfgs', 'liblinear']}
    model = tune_model(LogisticRegression(max_iter=300), param_grid, X_train, y_train)
    print("Logistic Regression Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "Logistic Regression")

def svm_model(X_train, y_train, X_test, y_test):
    param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
    model = tune_model(SVC(), param_grid, X_train, y_train)
    print("SVM Model Metrics:")
    print_metrics(y_train, model.predict(X_train), "Train")
    print_metrics(y_test, model.predict(X_test), "Test")
    train_pred=model.predict(X_train)
    test_pred=model.predict(X_test)
    performance_table(y_train, train_pred, y_test, test_pred, "SVM")

# Master function to run all models
def run_all_models(X_train, y_train, X_test, y_test):
    models = {
        'KNN': knn_model,
        'Random Forest': random_forest_model,
        'Decision Tree': decision_tree_model,
        'Naive Bayes': naive_bayes_model,
        'MLP': mlp_model,
        'XGBoost': xgboost_model,
        'AdaBoost': adaboost_model,
        'Logistic Regression': logistic_regression_model,
        'SVM': svm_model
    }
    for name, model_func in models.items():
        print(f"Running {name} model...")
        model_func(X_train, y_train, X_test, y_test)

# Input file path
input_file = '/content/drive/MyDrive/Colab Notebooks/ML_FINAL_DATASET_CONFIDENCE.xlsx'

# Load the dataset
df = pd.read_excel(input_file)

# Generate RoBERTa embeddings for the "answer" column
X = generate_roberta_embeddings(df['answer'])

# Apply PCA for dimensionality reduction
X_pca, pca_model = apply_pca(X, variance_threshold=0.95)

# Modify y to make sure the labels start from 0
y = df['confidence'].apply(lambda x: x + 1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)

# Run all models and print metrics
run_all_models(X_train, y_train, X_test, y_test)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Number of components selected: 135
Running KNN model...
Best Params: {'weights': 'uniform', 'n_neighbors': 14}
KNN Model Metrics:
Train Accuracy: 0.7185
Train Precision: 0.6864
Train Recall: 0.7185
Train F1 Score: 0.6721
Train Confusion Matrix:
[[  2  11  19]
 [  1  44  87]
 [  2  14 296]]

Test Accuracy: 0.7317
Test Precision: 0.6584
Test Recall: 0.7317
Test F1 Score: 0.6901
Test Confusion Matrix:
[[  0   7   6]
 [  0  11  30]
 [  0  12 139]]


Performance Table for KNN:
Metric              Value     
------------------------------
Train Accuracy      71.85%
Test Accuracy       73.17%
Train-Test Gap      1.32%
ROC-AUC             0.61
Overfitting         No


Running Random Forest model...
Best Params: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 30}
Random Forest Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7415
Test Precisi

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 27 is smaller than n_iter=30. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'adam', 'hidden_layer_sizes': (100,), 'activation': 'relu'}
MLP Model Metrics:
Train Accuracy: 0.9916
Train Precision: 0.9917
Train Recall: 0.9916
Train F1 Score: 0.9916
Train Confusion Matrix:
[[ 32   0   0]
 [  0 128   4]
 [  0   0 312]]

Test Accuracy: 0.7317
Test Precision: 0.7272
Test Recall: 0.7317
Test F1 Score: 0.7294
Test Confusion Matrix:
[[  2   8   3]
 [  4  18  19]
 [  5  16 130]]


Performance Table for MLP:
Metric              Value     
------------------------------
Train Accuracy      99.16%
Test Accuracy       73.17%
Train-Test Gap      25.99%
ROC-AUC             0.70
Overfitting         Yes


Running XGBoost model...
Best Params: {'subsample': 1.0, 'n_estimators': 100, 'learning_rate': 0.2}
XGBoost Model Metrics:
Train Accuracy: 0.9958
Train Precision: 0.9958
Train Recall: 0.9958
Train F1 Score: 0.9958
Train Confusion Matrix:
[[ 32   0   0]
 [  0 130   2]
 [  0   0 312]]

Test Accuracy: 0.7220
Test Precision: 0.6577
Test Recall: 0.7220
Test F

...

Best Params: {'n_estimators': 100, 'learning_rate': 0.1}
AdaBoost Model Metrics:
Train Accuracy: 0.6954
Train Precision: 0.6201
Train Recall: 0.6954
Train F1 Score: 0.6290
Train Confusion Matrix:
[[  0  14  18]
 [  0  31 101]
 [  0  12 300]]

Test Accuracy: 0.7171
Test Precision: 0.6328
Test Recall: 0.7171
Test F1 Score: 0.6671
Test Confusion Matrix:
[[  0   5   8]
 [  0   8  33]
 [  0  12 139]]


Performance Table for Ada Boost:
Metric              Value     
------------------------------
Train Accuracy      69.54%
Test Accuracy       71.71%
Train-Test Gap      2.17%
ROC-AUC             0.57
Overfitting         No


Running Logistic Regression model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=30. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 10}
Logistic Regression Model Metrics:
Train Accuracy: 0.8340
Train Precision: 0.8360
Train Recall: 0.8340
Train F1 Score: 0.8245
Train Confusion Matrix:
[[ 18   5   9]
 [  1  80  51]
 [  0  13 299]]

Test Accuracy: 0.7512
Test Precision: 0.7282
Test Recall: 0.7512
Test F1 Score: 0.7369
Test Confusion Matrix:
[[  2   7   4]
 [  2  17  22]
 [  2  14 135]]


Performance Table for Logistic Regression:
Metric              Value     
------------------------------
Train Accuracy      83.40%
Test Accuracy       75.12%
Train-Test Gap      8.28%
ROC-AUC             0.68
Overfitting         No


Running SVM model...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 6 is smaller than n_iter=30. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Params: {'kernel': 'rbf', 'C': 10}
SVM Model Metrics:
Train Accuracy: 0.9895
Train Precision: 0.9897
Train Recall: 0.9895
Train F1 Score: 0.9894
Train Confusion Matrix:
[[ 32   0   0]
 [  0 127   5]
 [  0   0 312]]

Test Accuracy: 0.7561
Test Precision: 0.7323
Test Recall: 0.7561
Test F1 Score: 0.7426
Test Confusion Matrix:
[[  2   7   4]
 [  5  16  20]
 [  2  12 137]]


Performance Table for SVM:
Metric              Value     
------------------------------
Train Accuracy      98.95%
Test Accuracy       75.61%
Train-Test Gap      23.34%
ROC-AUC             0.70
Overfitting         Yes


